In [1]:
# Add parent directory
from sys import path as sysPath
sysPath.append('../')

# Add custom modules
from Support_Code import general_module as gm
from Support_Code import info_module as im
from Image_Creator import main_image_creator as ic
from Score_Analysis import main_score_analysis as sa

sysPath.append('../Machine_Score/')
from Machine_Score import main_machine_score as ms
import main_SIMR as simr


# General modules
from os import path
import numpy as np
from matplotlib import pyplot as plt

if 'Machine_Compare' not in sysPath:
    sysPath.append('Machine_Compare')

from Image_Creator import main_image_creator as ic
import main_SIMR as simr

sa.test()
gm.test()
im.test()
ic.test()
ms.test()
simr.test()

SA: Hi!  You're in Matthew's Main program for score analysis!
GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew Ogden's information module for SPAM
IC: Hi!  You're in Matthew's main code for all things image creation.
MS: Hi!  You're in Matthew's SIMR module for all things machine scoring images
SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions


In [7]:
tDir = '../../targetDir/'
print(gm.validPath(tDir))
tInfo = im.target_info_class( targetDir = tDir, printAll=False)

if tInfo.status == False:
    print("WARNING!")

scores = tInfo.getScores()
print(scores)

rId = '00001'
rInfo = tInfo.getRunInfo(rID=rId, printAll = False)
rInfo.link_tInfo = tInfo
print(rInfo.status)

/mnt/c/Users/ogden/OneDrive/Desktop/galStuff/targetDir
      run_id  zoo_merger_score   test_v0  test_perturbation  base0_corr  \
0     r00000          0.945946  0.839341           0.798118         NaN   
1     r00001          0.941176  0.737633           0.716723    0.737633   
2     r00002          0.918919  0.793071           0.713520    0.793071   
3     r00003          0.916667  0.764586           0.689120    0.764586   
4     r00004          0.916667  0.730096                NaN    0.730096   
...      ...               ...       ...                ...         ...   
1288  r01288          0.000000  0.797219           0.954986    0.797219   
1289  r01289          0.000000  0.775788           0.992380    0.775788   
1290  r01290          0.000000  0.831271           0.993772    0.831271   
1291  r01291          0.000000  0.818303           0.948159    0.818303   
1292  r01292          0.000000  0.840964           0.976869    0.840964   

      base0_diff  base0_diff_scale  base0_ov

In [8]:
#tInfo.gatherRunInfos()
print(tInfo.getScores())
tInfo.updateScores()
print(tInfo.getScores())

      run_id  zoo_merger_score   test_v0  test_perturbation  base0_corr  \
0     r00000          0.945946  0.839341           0.798118         NaN   
1     r00001          0.941176  0.737633           0.716723    0.737633   
2     r00002          0.918919  0.793071           0.713520    0.793071   
3     r00003          0.916667  0.764586           0.689120    0.764586   
4     r00004          0.916667  0.730096                NaN    0.730096   
...      ...               ...       ...                ...         ...   
1288  r01288          0.000000  0.797219           0.954986    0.797219   
1289  r01289          0.000000  0.775788           0.992380    0.775788   
1290  r01290          0.000000  0.831271           0.993772    0.831271   
1291  r01291          0.000000  0.818303           0.948159    0.818303   
1292  r01292          0.000000  0.840964           0.976869    0.840964   

      base0_diff  base0_diff_scale  base0_overlap0  base0_bcorr0  \
0            NaN               

In [13]:

# Process target directory
def simr_target( arg=gm.inArgClass(), tInfo = None ):

    tDir = arg.targetDir
    printBase = arg.printBase
    printAll = arg.printAll

    if printBase:
        print("SIMR: pipelineTarget: input")
        print("\t - tDir: %s" % tDir )
        print("\t - tInfo: %s" % type(tInfo) )

    # Check if given a target
    if tInfo == None and tDir == None:
        print("SIMR: WARNING: pipelineTarget")
        print("\t - Please provide either target directory or target_info_class")
        return

    # Read target directory if location given. 
    elif tInfo == None:
        tInfo = im.target_info_class( targetDir=tDir, \
                printBase = printBase, printAll=printAll, \
                newInfo = arg.get('newInfo',False), \
                newRunInfos = arg.get('newRunInfos',False), )

    if printBase:
        print("SIMR: pipelineTarget status:")
        print("\t - tInfo.status: %s" % tInfo.status )

    # Check if valid directory
    if tInfo.status == False:
        print("SIMR: WARNING: pipelineTarget:  Target Info status bad")
        return

    if arg.get('printParam', False):
        tInfo.printParams()

    # Gather scores if called for
    if arg.get('update',False):
        tInfo.gatherRunInfos()
        tInfo.updateScores()
        tInfo.saveInfoFile()

    # Create new files/scores if called upon
    if arg.get('newAll') or arg.get('newScore') :
        new_target_scores( tInfo, arg )


def new_target_scores( tInfo, arg ):

    printBase = arg.printBase
    printAll = arg.printAll
    # Each target should have unique score params?
    paramLoc = arg.get('paramLoc',None)

    if printBase:
        print("SIMR: newTargetScores:")
        print("\t - tInfo: %s" % tInfo.status )
        im.tabprint('paramLoc: %s'%paramLoc)

    # Check if new parameter file is given
    if not gm.validPath(paramLoc):
        if printBase:
            print("SIMR: WARNING: new_target: param location not valid")
            gm.tabprint('paramLoc: %s',paramLoc)
        return
            
    # Read parameter file
    else:
        pClass = im.group_score_parameter_class(paramLoc)
        if pClass.status:
            params = pClass.get('group',None)
            del pClass

    # Check for valid parameter file
    if params == None:
        if printBase:
            print("SIMR: WARNING: Target_New: Failed to load parameter class")
            gm.tabprint('paramLoc: %s',paramLoc)
        return

    
    runDicts = tInfo.getAllRunDicts()
    
    # Prep arguments
    runArgs = gm.inArgClass()
    runArgs.setArg('printBase', False)
    runArgs.setArg('printAll', printAll)
    runArgs.setArg('newScore', True)
    runArgs.setArg('tInfo', tInfo)
    runArgs.setArg('scoreParams', params)
    runArgs.setArg('overWrite', arg.get('overWrite',False))
    
    # Prepare parallel class

    ppClass = gm.ppClass( arg.nProc, printProg=True )

    argList = [ dict( arg = runArgs, rDir=rDir, ) for rDir in tInfo.iter_runs() ]

    ppClass.loadQueue( simr.simr_run, argList )

    # Do parallel
    ppClass.runCores()

    tInfo.gatherRunInfos()
    tInfo.updateScores()
    tInfo.saveInfoFile()

# End processing target dir

In [15]:
pLoc = gm.validPath('../param/param_base0_scores.json')
print(pLoc)

tArgs = gm.inArgClass()
tArgs.setArg('paramLoc',pLoc)
tArgs.setArg('newScore',True)
tArgs.setArg('overWrite',True)
tArgs.setArg('nProc',3)


/mnt/c/Users/ogden/OneDrive/Desktop/galStuff/galaxyJSPAM/param/param_base0_scores.json


In [16]:
import timeit

start = timeit.default_timer()

simr_target( arg = tArgs, tInfo = tInfo )

stop = timeit.default_timer()

time = (stop - start) 

print('Time_per_target: ', time) 

SIMR: pipelineTarget: input
	 - tDir: None
	 - tInfo: <class 'Support_Code.info_module.target_info_class'>
SIMR: pipelineTarget status:
	 - tInfo.status: True
SIMR: newTargetScores:
	 - tInfo: True
	 - paramLoc: /mnt/c/Users/ogden/OneDrive/Desktop/galStuff/galaxyJSPAM/param/param_base0_scores.json
Process-19 - queue empty
Process-20 - queue empty
Process-18 - queue empty
Process-23 - queue empty
Process-22 - queue emptyProcess-21 - queue empty

{'machine_scores': {'run_id': 1293, 'test_v0': 1293, 'test_perturbation': 1289, 'base0_corr': 1292, 'base0_diff': 1292, 'base0_diff_scale': 1292, 'base0_overlap0': 1292, 'base0_bcorr0': 1292, 'base0_diff_scale1': 1292, 'base0_overlap1': 1292, 'base0_overlap2': 1292, 'base0_overlap3': 1292, 'base0_bcorr1': 1292, 'base0_bcorr2': 1292, 'base0_bcorr3': 1292, 'base0_diff_scale2': 1292}}
Time_per_target:  32.31283220000023


In [ ]:
print(tInfo.getScores())
tInfo.gatherRunInfos()
tInfo.updateScores()
print(tInfo.getScores())